In [76]:
import neurokit2 as nk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
#警告隱藏
warnings.filterwarnings("ignore")

In [47]:
#胸之呼吸

In [77]:
data = []
for i in range(1, 31):
    filename = f"P1-V{i}.csv"
    data.append(pd.read_csv(filename))

rsp_chest = [data[i]["D: Resp"] for i in range(30)]

# nk.signal_plot(rsp_chest[0], sampling_rate=16)

cleaned_chest = [nk.rsp_clean(rsp_chest[i], sampling_rate=16) for i in range (30)]

In [78]:
df_list = []
peaks_dicts = []

for i in range (30):
    df, peaks_dict = nk.rsp_peaks(cleaned_chest[i])
    df_list.append(df)
    peaks_dicts.append(peaks_dict)
    
info = [nk.rsp_fixpeaks(peaks_dict) for peaks_dict in peaks_dicts]

formatted = [nk.signal_formatpeaks(info[i], desired_length=len(cleaned_chest[i]),peak_indices=info[i]["RSP_Peaks"]) for i in range (30)]


In [79]:
info_list = []
event_list = []
epochs_list = []
df_list2 = []
result_list = []

for i in range (30):
    df, info = nk.bio_process(rsp= data[i]["D: Resp"], sampling_rate=16)
    df_list2.append(df)
    info_list.append(info)
    
    event = nk.events_find(data[i]["participant"], threshold_keep='below')
    event_list.append(event)
    
    epoch = nk.epochs_create(df_list2[i], event_list[i], sampling_rate=16)
    epochs_list.append(epoch)


In [80]:
#圖片避難區
nk.signal_plot(pd.DataFrame({"RSP_Raw":rsp_chest[0], "RSP_Clean": cleaned_chest[0]}), sampling_rate=16, subplots=True)
Chest_peaks = nk.events_plot(peaks_dicts[0]['RSP_Peaks'], cleaned_chest[0])
# Extract rate
rsp_chest_rate = nk.rsp_rate(cleaned_chest[0], peaks_dicts[0], sampling_rate=16)

# Visualize
nk.signal_plot(rsp_chest_rate, sampling_rate=16)
plt.ylabel('Breaths Per Minute')

rrv = nk.rsp_rrv(rsp_chest_rate, info_list[0], sampling_rate=16, show=True) #傅立葉分析
#需要看圖片把下面這一行刪掉
plt.close('all')

In [81]:
result_list = []

for i in range(30):
    try:
        result = nk.rsp_eventrelated(epochs_list[i])
        result_list.append(result)
    except:
        print(f"幹!epochs_list[{i}]出錯了，跳過跳過，csv中第{i+1}行下面原本需要另一個資料")

幹!epochs_list[22]出錯了，跳過跳過，csv中第23行下面原本需要另一個資料
幹!epochs_list[23]出錯了，跳過跳過，csv中第24行下面原本需要另一個資料


In [82]:
result_df = pd.concat(result_list, axis=0)
result_df.to_csv("P1_胸式呼吸_謝謝冠哲.csv", index=False)